In [6]:
import cv2 as cv
import numpy as np
import os
import keras
import copy

#Loading the image
img = cv.imread('Sample Sudoku Images/photo1.jpg')
# img = cv.imread('Sudoku/Sample Sudoku Images/photo2.jpg')
# cv.imshow('Original Image', img)
# cv.waitKey(0)

#Converting to Gray Scale
img_gray=cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# cv.imshow('Gray Scale Image', img_grey)
# cv.waitKey(0)

#Blur the image to reduce Noise
img_blur = cv.GaussianBlur(img_gray, (5,5), cv.BORDER_DEFAULT)
# cv.imshow('Blurred Image', img_blur)
# cv.waitKey(0)

#Thresholding the image
img_adaptive_thresh = cv.adaptiveThreshold(img_blur, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 13, 10)
# cv.imshow('Thresholded(Adaptive) Image', img_adaptive_thresh)
# cv.waitKey(0)

#Inverting the thresholded image
img_inverted = cv.bitwise_not(img_adaptive_thresh)
# cv.imshow('Inverted Image', img_inverted)
# cv.waitKey(0)

#Dilating the inverted image
img_dilated = cv.dilate(img_inverted, (3,3), iterations = 2)
# cv.imshow('Dilated Image', img_dilated)
# cv.waitKey(0)

#Finding Contours
contour, hierarchy = cv.findContours(img_dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

#Finding the Contour with Largest Area
largest = np.array([])
max_area = 0
for i in contour:
    area = cv.contourArea(i)
    perimeter = cv.arcLength(i, True)
    #Grid Extraction using Ramer–Douglas–Peucker algorithm
    sides = cv.approxPolyDP(i , 0.02*perimeter, True)
    if area > max_area and len(sides) == 4:
        largest = sides
        max_area = area
largest = largest.reshape(4, 2)

#Reframing largest for warping
reframed_largest = np.zeros((4,2),dtype = np.int32)
add = largest.sum(1)
diff = np.diff(largest, axis = 1)

reframed_largest[0] = largest[np.argmin(add)] #top left corner
reframed_largest[3] = largest[np.argmax(add)] #bottom right corner
reframed_largest[1] = largest[np.argmin(diff)] #top right corner
reframed_largest[2] = largest[np.argmax(diff)] #bottom left corner

points_orig = np.float32(reframed_largest)
points_new = np.float32([[0,0],[450,0],[0,450],[450,450]])
warp_matrix = cv.getPerspectiveTransform(points_orig, points_new)
img_warp_bgr = cv.warpPerspective(img, warp_matrix, (450,450))
img_warp = cv.cvtColor(img_warp_bgr, cv.COLOR_BGR2GRAY)
# cv.imshow('Warped Image bgr', img_warp_bgr)
# cv.imshow('Warped Image', img_warp)
# cv.waitKey(0)

#Applying Adaptive Thresholding to the warped image and then inverting it
img_warp_adaptive_thresh = cv.adaptiveThreshold(img_warp, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 13, 10)

img_warp_inverted_nondilated = cv.bitwise_not(img_warp_adaptive_thresh)
img_warp_inverted = cv.dilate(img_warp_inverted_nondilated, (3,3), iterations = 2)
# cv.imshow('Warped, Thresholded and Inverted Image', img_warp_inverted)
# cv.waitKey(0)

#Extracting Each Cells
extracted_cells = []
l=[]
for i in range(1,10):
    for j in range(1,10):
        new_image = img_warp_inverted[(i-1)*50:i*50, (j-1)*50:j*50]
        new_image = cv.resize(new_image, (28, 28), interpolation= cv.INTER_LINEAR)
        l.append(np.sum(new_image == 0))
        extracted_cells.append(new_image)

extracted_cells = np.array(extracted_cells)
l=np.array(l)
l = l.reshape((9,9))
extracted_cells = extracted_cells.reshape((81,28,28,1))

model = keras.models.load_model('model.h5')
def predict(img):
    image = img.copy()
#     image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
#     image = cv.threshold(image, 140, 255, cv.THRESH_BINARY)[1]
    image = cv.resize(image, (28, 28))
    # display_image(image)
    image = image.astype('float32')
    image = image.reshape(1, 28, 28, 1)
    image /= 255

    # plt.imshow(image.reshape(28, 28), cmap='Greys')
    # plt.show()
    
    pred = model.predict(image.reshape(1, 28, 28, 1), batch_size=1)
    return pred.argmax()

unsolved = np.zeros((81,1))
for i in range(81):
    if np.sum(extracted_cells[i] == 0)>550:
        unsolved[i,0] = 0
    else:
        unsolved[i,0] = predict(extracted_cells[i])
#         backup = 
unsolved = np.reshape(unsolved,(9,-1))
unsolved_copy = copy.deepcopy(unsolved)

def find_empty(board):
    for i in range(9):
        for j in range(9):
            if board[i,j]==0:
                return (i,j)
    return None

def valid(board, n, row, col):
    if n in board[row]:
        return False
    elif n in board[:,col]:
        return False
    elif n in board[3*(row//3):3*(row//3+1),3*(col//3):3*(col//3+1)]:
        return False
    return True

def solve(board):
    if find_empty(board):
        row, col = find_empty(board)
    else:
        return True
    
    for n in range(1,10):
        if valid(board, n, row, col):
            board[row][col] = n
            if solve(board):
                return True
            board[row][col] = 0
    return False

solve(unsolved)

unsolved = np.array(unsolved, dtype=int)

def displayNumbers(img, numbers, color=(0, 0, 255)):
    W = int(img.shape[1]/9)
    H = int(img.shape[0]/9)
    for i in range (9):
        for j in range (9):
            if numbers[(j*9)+i] !=0:
                cv.putText(img, str(numbers[(j*9)+i]), (i*W+int(W/2)-int((W/4)), int((j+0.7)*H)), cv.FONT_HERSHEY_COMPLEX, 1, color, 2, cv.LINE_AA)
    return img

def get_InvPerspective(img, masked_num, height = 900, width = 900):
    """Takes original image as input"""
#     pts1 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
#     pts2 = np.float32([location[0], location[3], location[1], location[2]])
    points_new = np.float32(reframed_largest)
    points_orig = np.float32([[0,0],[450,0],[0,450],[450,450]])
    # Apply Perspective Transform Algorithm
    matrix = cv.getPerspectiveTransform(points_orig, points_new)
    result = cv.warpPerspective(masked_num, matrix, (img.shape[1], img.shape[0]))
    return result

binArr = np.where(np.array(unsolved_copy.flatten())>0, 0, 1)
# # get only solved numbers for the solved board
flat_solved_board_nums = unsolved.flatten()*binArr
# create a mask
mask = np.zeros_like(img_warp_bgr)
solved_board_mask = displayNumbers(mask, flat_solved_board_nums)
# cv.imshow("Solved Mask", solved_board_mask)
# cv.waitKey(0)

# Get inverse perspective
inv = get_InvPerspective(img, solved_board_mask)
inv_white = get_InvPerspective(img, displayNumbers(mask, flat_solved_board_nums, color=(255, 255, 255)))

combined_sub = cv.addWeighted(img, 1, inv_white, -1, 0)

combined = cv.addWeighted(combined_sub, 1, inv, 1, 0)
cv.imshow("Final result", combined)


cv.waitKey(0)

1/1 [==============================] - 0s 32ms/step


-1